**About Feature**

---
- **شماره مشتری**: Customer ID
- **ِتاریخ**: Date
- **نام کالا**: Product name
- **stock_code**: stock_code
- **quantity**: Purchase Quantity
- **unit_price**:Price of every product


## **Importing necessary libraries**

In [1]:
import pandas as pd 
from apyori import apriori
import numpy as np

# 1.Overview of Data

In [3]:
data = pd.read_excel(r'C:\Users\ASUS\Desktop\project\apriori\Apriori.xlsx')

In [5]:
data.head(10)

,شماره مشتری,تاریخ,stock_code,نام کالا,quantity,unit_price
0,224356,12/11/2018,5585,دلستر خانواده,4.0,30000
1,224356,12/11/2018,5586,دلستر قوطی,4.0,30000
2,224356,12/16/2018,5370,شیر 250 cc,4.0,14000
3,224356,12/23/2018,3531,شیر قهوه موکا,2.0,14000
4,224356,12/23/2018,8596,آب پرتقال 250 cc,3.0,14000
5,224356,12/23/2018,8775,آب آلبالو 250 cc,3.0,14000
6,224356,12/30/2018,5370,شیر 250 cc,6.0,14000
7,224356,12/30/2018,8596,آب پرتقال 250 cc,3.0,14000
8,224356,12/30/2018,8775,آب آلبالو 250 cc,3.0,14000
9,224356,1/13/2019,5585,دلستر خانواده,2.0,30000


In [4]:
n_rows, n_columns = data.shape
print(f"Number of columns: {n_columns} columns\nNumber of rws: {n_rows} rows")

Number of columns: 6 columns
Number of rws: 412253 rows


In [29]:
data.dtypes

CID             int64
date           object
stock_name     object
quantity      float64
unit_price      int64
dtype: object

In [31]:
print('Known observations: {}\nUnique observations: {}'.format(len(data.index),len(data.drop_duplicates().index)))
if len(data.index)==len(df.drop_duplicates().index):
    print("there is no duplicate")
else:
    print("there is {} duplicates.".format(len(data.index) - len(data.drop_duplicates().index)))

Known observations: 412253
Unique observations: 412253
there is 0 duplicates.


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412253 entries, 0 to 412252
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   CID         412253 non-null  int64  
 1   date        412253 non-null  object 
 2   stock_name  412253 non-null  object 
 3   quantity    412253 non-null  float64
 4   unit_price  412253 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 15.7+ MB


In [40]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
CID,412253.0,2.651539e+07,1.984150e+07,224356.0,4706288.0,44337061.0,44556765.0,49431856.0
quantity,412253.0,3.444537e+00,3.274752e+00,-97.0,2.0,4.0,5.0,300.0
unit_price,412253.0,1.425041e+04,1.985914e+03,14000.0,14000.0,14000.0,14000.0,30000.0


In [41]:
print("**Unique Values By Features**")
for feature in data.columns:
    uniq = np.unique(df[feature])
    print(feature.ljust(21),len(uniq))

**Unique Values By Features**
CID                   8961
date                  335
stock_name            8
quantity              77
unit_price            2


In [42]:
cat_cols = list(set(df.columns) - set(df._get_numeric_data().columns))
num_cols = list(set(df._get_numeric_data().columns))
cat_cols,num_cols

(['stock_name', 'date'], ['unit_price', 'quantity', 'CID'])

# 2.Analysis

<font color=#0b5bdb>**1. Columns that are Farsi should be translated into English for better interpretation.**</font>

In [6]:
data.rename(columns={
    'شماره مشتری': 'CID', 
    'تاریخ': 'date',
    'نام کالا': 'stock_name'
}, inplace=True)

<font color=#0b5bdb>**2. We don't seem to have null value, but let's check and see that we really don't have any**</font>

In [7]:
data.sort_values('date')

,CID,date,stock_code,stock_name,quantity,unit_price
273886,44549570,1/1/2019,3531,شیر قهوه موکا,-1.0,14000
101722,4706223,1/1/2019,3531,شیر قهوه موکا,5.0,14000
287377,44552996,1/1/2019,3531,شیر قهوه موکا,1.0,14000
287378,44552996,1/1/2019,8596,آب پرتقال 250 cc,2.0,14000
287379,44552996,1/1/2019,8775,آب آلبالو 250 cc,3.0,14000
...,...,...,...,...,...,...
31564,4702549,9/8/2019,8775,آب آلبالو 250 cc,5.0,14000
391308,44565004,9/8/2019,8596,آب پرتقال 250 cc,5.0,14000
391309,44565004,9/8/2019,8775,آب آلبالو 250 cc,5.0,14000
254075,44539921,9/8/2019,8775,آب آلبالو 250 cc,3.0,14000


In [8]:
data.sort_values('CID')

,CID,date,stock_code,stock_name,quantity,unit_price
0,224356,12/11/2018,5585,دلستر خانواده,4.0,30000
20,224356,8/27/2019,3531,شیر قهوه موکا,2.0,14000
21,224356,8/27/2019,5370,شیر 250 cc,2.0,14000
22,224356,8/27/2019,8596,آب پرتقال 250 cc,2.0,14000
24,224356,9/24/2019,7178,آب انبه 200 cc,1.6,14000
...,...,...,...,...,...,...
412229,49332232,10/27/2019,8596,آب پرتقال 250 cc,5.0,14000
412230,49332232,10/27/2019,8775,آب آلبالو 250 cc,5.0,14000
412231,49332232,11/5/2019,3531,شیر قهوه موکا,4.0,14000
412233,49332232,11/5/2019,8596,آب پرتقال 250 cc,4.0,14000


We can see that there is no additional sign before and after the first and last date

<font color=#0b5bdb>**3. find out whether stock_code and stock_name are equal.**</font>

In [9]:

for code in data.stock_code.unique():
    temp=data[data.stock_code==code]
    if temp['stock_name'].nunique() != 1:
        print(code, temp['stock_name'].unique())

In [10]:
data.drop('stock_code', axis=1, inplace=True)

So we can eliminate  stock_code they are equal

<font color=#0b5bdb>**3. We can see negative values ​​in quantity, This means that this product has been returned and we do not need this information at the moment .**</font>

In [33]:
display(data[data.quantity<0])

,CID,date,stock_name,quantity,unit_price
14,224356,3/2/2019,شیر قهوه موکا,-6.0,14000
15,224356,3/16/2019,شیر 250 cc,-2.0,14000
16,224356,3/16/2019,آب آلبالو 250 cc,-2.0,14000
26,224356,9/25/2019,شیر قهوه موکا,-1.0,14000
27,224356,10/14/2019,شیر قهوه موکا,-1.0,14000
...,...,...,...,...,...
412242,49332232,12/11/2019,شیر قهوه موکا,-3.0,14000
412243,49332232,12/11/2019,آب پرتقال 250 cc,-2.0,14000
412246,49332232,12/14/2019,شیر قهوه موکا,-1.0,14000
412247,49332232,12/14/2019,آب پرتقال 250 cc,-1.0,14000


In [11]:
df=data[data.quantity>0]

<font color=#0b5bdb>**4. Find the customer's shopping cart in each purchase .**</font>

In [12]:

data_temp = df.copy()
data_temp['stock_name']=data_temp['stock_name']+','
grouped=data_temp.groupby(['CID', 'date']).agg({'stock_name': 'sum'})
records = []
for line in grouped['stock_name']:
    items = line.split(',')
    temp = []
    for item in items:
        if item.strip() != '':
            temp.append(item)
    records.append(temp)
records

[['دلستر خانواده', 'دلستر قوطی'],
 ['شیر قهوه موکا'],
 ['شیر قهوه موکا',
  'شیر 250 cc',
  'آب انبه 200 cc',
  'آب پرتقال 250 cc',
  'آب آلبالو  250 cc'],
 ['دلستر خانواده', 'دلستر قوطی'],
 ['شیر 250 cc'],
 ['شیر قهوه موکا', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['شیر 250 cc', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['دلستر قوطی'],
 ['دلستر خانواده'],
 ['دلستر قوطی'],
 ['دلستر خانواده', 'دلستر قوطی'],
 ['شیر قهوه موکا', 'شیر 250 cc', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['آب انبه 200 cc', 'آب پرتقال 250 cc'],
 ['شیر 250 cc', 'آب پرتقال 250 cc'],
 ['آب آلبالو  250 cc'],
 ['شیر قهوه موکا', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['شیر 250 cc', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['شیر قهوه موکا', 'شیر 250 cc', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['آب انبه 200 cc'],
 ['شیر قهوه موکا', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['آب آلبالو  250 cc'],
 ['آب آلبالو  250 cc'],
 ['شیر 250 cc', 'آب پرتقال 250 cc', 'آب آلبالو  250 cc'],
 ['شیر 250 cc', 'آب پرتقال 2

## 2.apriori

In [24]:
result=apriori(records, min_support=0.1, min_lift=1.2, min_confidence=0.7)
for res in result:
    for item in res[2]:
        print(', '.join(list(item.items_base)) ,'---------->', ', '.join(list(item.items_add)))
        print('support:', res[1])
        print('confidence:', item.confidence)
        print('lift:', item.lift)
        print()
        print('------------------------------------------------------------')
        print()

آب آلبالو  250 cc, شیر 250 cc ----------> آب پرتقال 250 cc
support: 0.20453841246696436
confidence: 0.773184511506132
lift: 1.2899601105999818

------------------------------------------------------------

آب پرتقال 250 cc, شیر 250 cc ----------> آب آلبالو  250 cc
support: 0.20453841246696436
confidence: 0.833275580416811
lift: 1.2517271372781578

------------------------------------------------------------

شیر قهوه موکا, آب آلبالو  250 cc ----------> آب پرتقال 250 cc
support: 0.21501260670129713
confidence: 0.7566816335257645
lift: 1.2624271556739906

------------------------------------------------------------

شیر قهوه موکا, آب پرتقال 250 cc ----------> آب آلبالو  250 cc
support: 0.21501260670129713
confidence: 0.8320598123809748
lift: 1.2499008388975783

------------------------------------------------------------

شیر قهوه موکا, شیر 250 cc ----------> آب آلبالو  250 cc
support: 0.17957410613931166
confidence: 0.816446605159936
lift: 1.22644704325323

-----------------------------